#everpeak_analysis

# Análisis Estadístico y Preparación de Datos - EverPeak Holdings

## Descripción del proyecto
Este notebook documenta el proceso completo de análisis estadístico y preparación de datos
para EverPeak Holdings como parte del proceso de integración con SilverBasket Retail Group.

## Objetivos del sprint
- Preparación de datos para análisis estadístico
- Cálculo de medidas estadísticas en columnas numéricas  
- Análisis de valores atípicos y segmentación
- Documentación del proceso de limpieza y análisis

## Dataset
- **Archivo:** everpeak_retail.csv → everpeak_clean.csv
- **Descripción:** Datos transaccionales de EverPeak Holdings

## Librerías para análisis de datos

In [ ]:
import pandas as pd
import numpy as np

## Librerías para visualización

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el dataset y revisar estructura general

In [ ]:

import pandas as pd

df = pd.read_csv("everpeak_retail.csv")
df.info()

## Conteo de valores faltantes por columna
payment_missing = df["payment_method"].isna().sum()
city_missing = df["city"].isna().sum()
state_missing = df["state"].isna().sum()

## Fechas sospechosas y montos extremos
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")

invalid_year_2026_count = (df["order_date"].dt.year == 2026).sum()
missing_order_date_count = df["order_date"].isna().sum()

## Cardinalidad en columnas de cliente
customer_id_unicos = df["customer_id"].nunique()
payment_unicos = df["payment_method"].nunique()
city_unicos = df["city"].nunique()
state_unicos = df["state"].nunique()


# Manejando valores ausentes o inválidos

In [ ]:
## Detectar patrones de missingness por grupo
missing_city_by_pay = df["city"].isna().groupby(df["payment_method"]).mean()
print(missing_city_by_pay)

##Comparar versiones de una columna numérica para decidir la estrategia final
before = df['order_value'].dropna().mean()
df["order_value_imputed"] = df['order_value'].fillna(df['order_value'].median())
after = df['order_value_imputed'].mean()

# Construyendo un pipeline completo para limpieza de datos

In [ ]:

## Funciones auxiliares
def reemplazar_sentinels(df, sentinels, numeric_cols):
    for col in numeric_cols:
        df[col] = df[col].replace(sentinels, pd.NA)
    return df

def rellenar_ausentes(df, cols_fill):
    for col in cols_fill:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col].fillna(df[col].mean(), inplace=True)
    return df

## Crear función pipeline
def limpiar_df(df):
    valores_erroneos = [-999, 999, 0, 1]
    columnas_numericas = ["customer_age", "price"]
    df = reemplazar_sentinels(df, valores_erroneos, columnas_numericas)
    df = rellenar_ausentes (df, columnas_numericas)
    return df

## Observar valores ausentes iniciales
print("Valores ausentes iniciales:")
print(df[["customer_age", "price"]].isna().sum())

## Aplicar pipeline completo
df = limpiar_df (df)
print("\nValores ausentes después del pipeline:")
print(df[["customer_age", "price"]].isna().sum())



#-----------------------------------------------------------------

# Descripción de distribuición de datos



In [3]:
## Promedio vs Mediana
print("Promedio de quantity: ", df['quantity'].mean())
print("Mediana de quantity: ", df['quantity'].median())
print("El promedio está afectado por los outliers o valores atípicos.")

## Resúmen Numérico
### Crear dataframes para cada categoría
df_ny = df[df['city'] == 'New York']
df_la = df[df['city'] == 'Los Angeles']

### Resumen de columnas numéricas
columnas_numericas = ['order_value', 'customer_age', 'price', 'quantity']

print('Resumen estadístico de la ciudad New York')
print(df_ny[columnas_numericas].describe())

print()
print('Resumen estadístico de la ciudad Los Angeles')
print(df_la[columnas_numericas].describe())

## Resumen de columnas categóricas
### Columnas categóricas
columnas_categoricas = ['payment_method', 'city']

### Filtra por categoría
df_toys = df[df['product_category'] =="Toys"]

### Resumen categórico Toys
print("Resumen categórico - Toys")
print(df_toys[columnas_categoricas].describe())

## Visualización de distribuición con Histogramas

### Graficar histograma
counts, bin_edges, _ = plt.hist(df['price'], bins=10, range=(0,1000), color='skyblue', edgecolor='black')

### Mostrar las marcas de los bins en el eje X
plt.xticks(bin_edges)

### Etiquetas y título del gráfico
plt.xlabel("Precio")
plt.ylabel("Cantidad")
plt.title("Distribución de Precios")
plt.show()

## Visualiación de distribuición con Boxplots

### Filtrar datos
df_fashion = df[df['product_category'] =='Fashion']
df_sports = df[df['product_category'] =='Sports']

### BoxPlot Categoría Fashion
sns.boxplot(x=df_fashion['order_value'], color='skyblue')
plt.xlabel("Gasto del Cliente")
plt.title("Boxplot de Gastos - Categoría Fashion")
plt.show()

### BoxPlot Categoría Sports
sns.boxplot(x=df_sports['order_value'], color='skyblue')
plt.xlabel("Gasto del Cliente")
plt.title("Boxplot de Gastos - Categoría Sports")
plt.show()

NameError: name 'df' is not defined

#-----------------------------------------------------------------

# Análisis de valores atípicos y segmentación

In [ ]:

## Detectar Outliers con IQR

### calcular Q1, Q3 e IQR
Q1 = df['order_value'].quantile(0.25)
Q3 = df['order_value'].quantile(0.75)
IQR = Q3 - Q1

### calcular límite inferior y superior
limite_inferior = Q1 - 1.5*IQR
limite_superior = Q3 + 1.5*IQR

### Mostrar resultados
print('Primer cuartil: ', Q1)
print('Tercer cuartil: ', Q3)
print('IQR: ', IQR)

print("\nRegistros abajo del límite inferior")
print(df[df['order_value'] < limite_inferior])

print("\nRegistros arriba del límite superior")
print(df[df['order_value'] > limite_superior])

# Z-Scores

### cálculo de la media
mean = df['order_value'].mean()

### cálculo de la desviación estándar
std = df['order_value'].std()

### Crea el z score
df['z'] = df['z'] = (df['order_value'] - mean) / std

### Calcula los valores extremos
print(df[df['z'].abs() > 3])

##Segmentación de Clientes usando if

### Calcular promedio y mediana
cantidad_promedio = df['quantity'].mean()
cantidad_mediana = df['quantity'].median()
print("Promedio:", cantidad_promedio)
print("Mediana:",cantidad_mediana)
print()

### Segmentación con media o promedio
if cantidad_promedio > 22:
	print("En promedio: volumen alto")
elif cantidad_promedio >= 10:
	print("En promedio: volumen medio")
else:
	print("En promedio: volumen bajo")

### Segmentación con mediana
if cantidad_mediana > 22:
	print("Según la mediana: volumen alto")
elif cantidad_promedio >= 10:
	print("Según la mediana: volumen medio")
else:
	print("Según la mediana: volumen bajo")

## Segementación con función Apply

### Crear función
 def classify_payment(row):
    card = row['payment_method']
    qty = row['quantity']

### Manejo de valores nulos/faltantes
    if pd.isna(card) or pd.isna(qty):
        return "Error en Datos"

### Segmentación de Altas Cantidades
    if qty > 22:
        if card == "credit_card" or card == "debit_card":
            return "card_high_volume"
        else:
            return "no_card_high_volume"

### Segmentación Bajas Cantidades
    elif qty <= 22:
        if card == "credit_card" or card == "debit_card":
            return "card_low_volume"
        else:
            return "no_card_low_volume"

### aplicar función y verificar cambios
df['payment_segment'] = df.apply(classify_payment, axis=1)
print(df['payment_segment'].value_counts())
